In [1]:
%env RAY_verbose_spill_logs=0
%env RAY_SESSION_DIR="/home/ray/"

env: RAY_verbose_spill_logs=0
env: RAY_SESSION_DIR="/home/ray/"


In [2]:
import ray
import os
from concurrent.futures import ThreadPoolExecutor
from minio import Minio
import urllib3
import uuid
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
import time
import pickle

# Defined the function that calculates the sum of squares
def calculate_sum_of_squares(n):
    return sum(i * i for i in range(1, n+1))


# Function to distribute the computation and write results
def distributed_task(worker_id, n, shared_storage_path):
    result = calculate_sum_of_squares(n)
    result_file = os.path.join(shared_storage_path, f"result_{worker_id}.txt")
    print("result-value:", result,"||", "result-file-path:", result_file)
    with open(result_file, "w") as f:
        f.write(str(result))
    return result_file


def main():
    # Initialize the RAY cluster
    ray.init(address="ray://kuberay-head-svc.kuberay:10001", ignore_reinit_error=True, )

    # Shared storage path
    shared_storage_path = "./shared/custompath/"
    result_file=""

    # Number of workers to use
    num_loops = 10
    total_sum = 0
        
    # Number to calculate sum of squares up to
    N = 100

    # Launch distributed tasks using ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=num_loops) as executor:
        task_futures = [executor.submit(distributed_task, i, N, shared_storage_path) for i in range(num_loops)]
    
    # Get results
    results = [future.result() for future in task_futures]
    # print(results)
    
    for i in range(0, int(len(results))):
        with open(results[i], "r") as f:
            total_sum += int(f.read())
            
    print("Total sum of squares:", total_sum)

    # Shutdown the RAY cluster
    ray.shutdown()
    
if __name__=="__main__":
    main()
    print("THE CUSTOM FILE PATH JOB END")

2023-08-16 11:02:23,943	INFO client_builder.py:237 -- Passing the following kwargs to ray.init() on the server: ignore_reinit_error


result-value: 338350 || result-file-path: ./shared/custompath/result_0.txt
result-value: 338350 || result-file-path: ./shared/custompath/result_1.txt
result-value: 338350 || result-file-path: ./shared/custompath/result_2.txt
result-value: 338350 || result-file-path: ./shared/custompath/result_3.txt
result-value: 338350 || result-file-path: ./shared/custompath/result_4.txt
result-value: 338350 || result-file-path: ./shared/custompath/result_5.txt
result-value: 338350 || result-file-path: ./shared/custompath/result_6.txt
result-value: 338350 || result-file-path: ./shared/custompath/result_7.txt
result-value: 338350 || result-file-path: ./shared/custompath/result_8.txt
result-value: 338350 || result-file-path: ./shared/custompath/result_9.txt
Total sum of squares: 3383500
THE CUSTOM FILE PATH JOB END
